In [1]:
import socket
import threading
import os
import struct

# -------- CONNECT --------
s = socket.socket()
s.connect(('localhost', 9996))

print("Connected to secure chat server")

# -------- RECEIVE THREAD --------
def receive():
    while True:
        try:
            data = s.recv(1024)

            if not data:
                print("Server disconnected")
                break

            data = data.decode()

            if data.startswith("MSG"):
                print(data[4:])

            elif data.startswith("File"):
                print(data)

        except Exception as e:
            print("Receive error:", e)
            break

threading.Thread(target=receive, daemon=True).start()

# -------- SEND LOOP --------
while True:
    try:
        type_ = input("Send message (m/M) or file (f/F)?: ")

        # -------- MESSAGE --------
        if type_.lower() == "m":
            msg = input("Client: ")
            s.send(f"MSG:{msg}".encode())

            if msg.lower() == "exit":
                print("Disconnected by client")
                s.close()
                break

        # -------- FILE --------
        elif type_.lower() == "f":
            filename = input("Filename: ")

            if not os.path.exists(filename):
                print("File not found")
                continue

            filesize = os.path.getsize(filename)

            # Send file header
            s.send(f"FILE:{os.path.basename(filename)}".encode())

            # Send file size
            s.send(struct.pack("!I", filesize))

            # Send file data in chunks
            with open(filename, "rb") as filep:
                while True:
                    data = filep.read(4096)
                    if not data:
                        break
                    s.send(data)

            print("File sent successfully")

        else:
            print("Invalid option")

    except Exception as e:
        print("Send error:", e)
        break


Connected to secure chat server
from ('127.0.0.1', 56182): hi


Send message (m/M) or file (f/F)?:  f
Filename:  Q3ex.pdf


File sent successfully
Receive error: [WinError 10054] An existing connection was forcibly closed by the remote host


Send message (m/M) or file (f/F)?:  m
Client:  exit


Send error: [WinError 10054] An existing connection was forcibly closed by the remote host
